In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
enron_data = pd.read_csv('C:\\Users\m_aminan\Downloads\emails.csv')


In [8]:
print(enron_data.head())

                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


In [9]:
print(enron_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB
None


In [11]:
print(enron_data.describe())

                         file  \
count                  517401   
unique                 517401   
top     allen-p/_sent_mail/1.   
freq                        1   

                                                  message  
count                                              517401  
unique                                             517401  
top     Message-ID: <18782981.1075855378110.JavaMail.e...  
freq                                                    1  


In [12]:
print(enron_data.isnull().sum())


file       0
message    0
dtype: int64


In [13]:
print(enron_data['message'][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [14]:
import email
from email.parser import Parser

def parse_email(text):
    # Parse the email content
    msg = email.message_from_string(text)

    # Extract headers
    from_ = msg['from']
    to_ = msg['to']
    subject_ = msg['subject']

    # Get email body
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                break
    else:
        body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')

    return {'from': from_, 'to': to_, 'subject': subject_, 'body': body}


In [15]:
# Apply the parsing function to each email message
parsed_emails = enron_data['message'].apply(parse_email)

# Convert the list of dictionaries to a DataFrame
parsed_emails_df = pd.DataFrame(list(parsed_emails))

# Concatenate with the original DataFrame
enron_data = pd.concat([enron_data, parsed_emails_df], axis=1)


In [16]:
# Display the first few rows of the new DataFrame
print(enron_data.head())
print("---------------------------------------------------")

# Explore the distribution of emails sent from different addresses
print(enron_data['from'].value_counts())
print("---------------------------------------------------")


# Basic EDA on the subject and body of the emails
print(enron_data['subject'].head())
print(enron_data['body'].head())


                       file  \
0     allen-p/_sent_mail/1.   
1    allen-p/_sent_mail/10.   
2   allen-p/_sent_mail/100.   
3  allen-p/_sent_mail/1000.   
4  allen-p/_sent_mail/1001.   

                                             message                     from  \
0  Message-ID: <18782981.1075855378110.JavaMail.e...  phillip.allen@enron.com   
1  Message-ID: <15464986.1075855378456.JavaMail.e...  phillip.allen@enron.com   
2  Message-ID: <24216240.1075855687451.JavaMail.e...  phillip.allen@enron.com   
3  Message-ID: <13505866.1075863688222.JavaMail.e...  phillip.allen@enron.com   
4  Message-ID: <30922949.1075863688243.JavaMail.e...  phillip.allen@enron.com   

                        to    subject  \
0     tim.belden@enron.com              
1  john.lavorato@enron.com        Re:   
2   leah.arsdall@enron.com   Re: test   
3    randall.gay@enron.com              
4     greg.piper@enron.com  Re: Hello   

                                                body  
0                       

In [17]:
import re

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Remove single character terms
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    return text


In [18]:
enron_data['cleaned_subject'] = enron_data['subject'].apply(clean_text)
enron_data['cleaned_body'] = enron_data['body'].apply(clean_text)


In [20]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Load stopwords once
english_stopwords = set(stopwords.words('english'))

def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in english_stopwords]
    return ' '.join(filtered_tokens)

# Apply the function
enron_data['processed_subject'] = enron_data['cleaned_subject'].apply(tokenize_and_remove_stopwords)
enron_data['processed_body'] = enron_data['cleaned_body'].apply(tokenize_and_remove_stopwords)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\m_aminan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\m_aminan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
print(enron_data[['processed_subject', 'processed_body']].head())

  processed_subject                                     processed_body
0                                                             forecast
1                    traveling business meeting takes fun trip espe...
2              test                             test successful way go
3                    randy send schedule salary level everyone sche...
4             hello                                  let shoot tuesday


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust 'max_features' as needed

# Apply it to both subjects and bodies of emails
tfidf_subject = tfidf_vectorizer.fit_transform(enron_data['processed_subject'])
tfidf_body = tfidf_vectorizer.fit_transform(enron_data['processed_body'])


In [26]:
tfidf_subject_df = pd.DataFrame(tfidf_subject.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_body_df = pd.DataFrame(tfidf_body.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


In [27]:
# Concatenate with the original DataFrame (if needed)
enron_data = pd.concat([enron_data, tfidf_subject_df, tfidf_body_df], axis=1)


In [28]:
print(enron_data.head())

                       file  \
0     allen-p/_sent_mail/1.   
1    allen-p/_sent_mail/10.   
2   allen-p/_sent_mail/100.   
3  allen-p/_sent_mail/1000.   
4  allen-p/_sent_mail/1001.   

                                             message                     from  \
0  Message-ID: <18782981.1075855378110.JavaMail.e...  phillip.allen@enron.com   
1  Message-ID: <15464986.1075855378456.JavaMail.e...  phillip.allen@enron.com   
2  Message-ID: <24216240.1075855687451.JavaMail.e...  phillip.allen@enron.com   
3  Message-ID: <13505866.1075863688222.JavaMail.e...  phillip.allen@enron.com   
4  Message-ID: <30922949.1075863688243.JavaMail.e...  phillip.allen@enron.com   

                        to    subject  \
0     tim.belden@enron.com              
1  john.lavorato@enron.com        Re:   
2   leah.arsdall@enron.com   Re: test   
3    randall.gay@enron.com              
4     greg.piper@enron.com  Re: Hello   

                                                body cleaned_subject  \
0      

In [30]:
print(enron_data.columns)


Index(['file', 'message', 'from', 'to', 'subject', 'body', 'cleaned_subject',
       'cleaned_body', 'processed_subject', 'processed_body',
       ...
       'would', 'www', 'xls', 'yahoo', 'year', 'years', 'yes', 'yesterday',
       'yet', 'york'],
      dtype='object', length=2010)


In [ ]:
from sklearn.cluster import KMeans

# Example: Applying K-Means to the TF-IDF features
text_data = enron_data['processed_body']  # or enron_data['processed_subject']
tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)

In [40]:
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(tfidf_matrix) 

C:\Users\m_aminan\AppData\Roaming\Python\Python38\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [41]:
# Add the cluster labels to your dataframe
enron_data['cluster'] = clusters

# Inspect a few emails from each cluster
print(enron_data[enron_data['cluster'] == 0].head())  # Change 0 to 1, 2, etc. for other clusters


                       file  \
0     allen-p/_sent_mail/1.   
1    allen-p/_sent_mail/10.   
2   allen-p/_sent_mail/100.   
3  allen-p/_sent_mail/1000.   
4  allen-p/_sent_mail/1001.   

                                             message                     from  \
0  Message-ID: <18782981.1075855378110.JavaMail.e...  phillip.allen@enron.com   
1  Message-ID: <15464986.1075855378456.JavaMail.e...  phillip.allen@enron.com   
2  Message-ID: <24216240.1075855687451.JavaMail.e...  phillip.allen@enron.com   
3  Message-ID: <13505866.1075863688222.JavaMail.e...  phillip.allen@enron.com   
4  Message-ID: <30922949.1075863688243.JavaMail.e...  phillip.allen@enron.com   

                        to    subject  \
0     tim.belden@enron.com              
1  john.lavorato@enron.com        Re:   
2   leah.arsdall@enron.com   Re: test   
3    randall.gay@enron.com              
4     greg.piper@enron.com  Re: Hello   

                                                body cleaned_subject  \
0      

In [43]:
from collections import Counter
import pandas as pd


def get_top_terms(data, cluster_number, num_terms=10):
    # Filter data for the specified cluster
    cluster_data = data[data['cluster'] == cluster_number]
    
    # Combine all text in the cluster
    all_text = ' '.join(cluster_data['processed_body'])  # Assuming 'processed_body' is the column with text data
    
    # Tokenize and count word frequency
    word_list = all_text.split()
    word_freq = Counter(word_list)
    
    # Get the most common terms
    common_terms = word_freq.most_common(num_terms)
    
    return common_terms



# Adjust 'num_clusters' to match the number of clusters you have
num_clusters = 2  # Example for 2 clusters

for cluster_num in range(num_clusters):
    print(f"Cluster {cluster_num}: {get_top_terms(enron_data, cluster_num)}")


Cluster 0: [('com', 1278636), ('enron', 1235099), ('please', 327547), ('http', 326603), ('subject', 321474), ('pm', 308480), ('energy', 304003), ('power', 297275), ('would', 293812), ('new', 260634)]
Cluster 1: [('ect', 1013368), ('enron', 633207), ('hou', 490812), ('subject', 125587), ('corp', 123684), ('cc', 122286), ('pm', 108772), ('ees', 92478), ('na', 79373), ('com', 72053)]


In [44]:
# Review a sample of emails from each cluster
for cluster_num in range(num_clusters):
    print(f"Cluster {cluster_num} Sample Emails:")
    sample_emails = enron_data[enron_data['cluster'] == cluster_num]['message'].sample(5)
    print(sample_emails)


Cluster 0 Sample Emails:
                                                  message  message   message
28526   Message-ID: <6909344.1075855873255.JavaMail.ev...      0.0  0.000000
187624  Message-ID: <24176954.1075847303095.JavaMail.e...      0.0  0.000000
368556  Message-ID: <23495623.1075852106674.JavaMail.e...      0.0  0.000000
436209  Message-ID: <22626670.1075862247543.JavaMail.e...      0.0  0.137718
185898  Message-ID: <30564396.1075847041114.JavaMail.e...      0.0  0.135062
Cluster 1 Sample Emails:
                                                  message  message  message
428543  Message-ID: <8349842.1075844527434.JavaMail.ev...      0.0      0.0
111326  Message-ID: <27302301.1075854327156.JavaMail.e...      0.0      0.0
301166  Message-ID: <4388697.1075846092676.JavaMail.ev...      0.0      0.0
195444  Message-ID: <33446796.1075847182975.JavaMail.e...      0.0      0.0
327178  Message-ID: <412513.1075843902241.JavaMail.eva...      0.0      0.0


In [45]:
# Experiment with different numbers of clusters
for k in range(2, 5):  # Trying with 2, 3, and 4 clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(tfidf_matrix)
    print(f"Top terms for {k} clusters:")
    for cluster_num in range(k):
        print(f"Cluster {cluster_num}: {get_top_terms(enron_data, cluster_num)}")


C:\Users\m_aminan\AppData\Roaming\Python\Python38\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Top terms for 2 clusters:
Cluster 0: [('com', 1278636), ('enron', 1235099), ('please', 327547), ('http', 326603), ('subject', 321474), ('pm', 308480), ('energy', 304003), ('power', 297275), ('would', 293812), ('new', 260634)]
Cluster 1: [('ect', 1013368), ('enron', 633207), ('hou', 490812), ('subject', 125587), ('corp', 123684), ('cc', 122286), ('pm', 108772), ('ees', 92478), ('na', 79373), ('com', 72053)]
Top terms for 3 clusters:
Cluster 0: [('com', 1278636), ('enron', 1235099), ('please', 327547), ('http', 326603), ('subject', 321474), ('pm', 308480), ('energy', 304003), ('power', 297275), ('would', 293812), ('new', 260634)]
Cluster 1: [('ect', 1013368), ('enron', 633207), ('hou', 490812), ('subject', 125587), ('corp', 123684), ('cc', 122286), ('pm', 108772), ('ees', 92478), ('na', 79373), ('com', 72053)]
Cluster 2: []
Top terms for 4 clusters:
Cluster 0: [('com', 1278636), ('enron', 1235099), ('please', 327547), ('http', 326603), ('subject', 321474), ('pm', 308480), ('energy', 3040

In [ ]:
print("kir")